In [22]:
sys.path.append("..")
import pandas as pd 
import numpy as np
from app.benchmarkers import UPSBenchmarker
experiment_home = "/mnt/d/Dropbox/MassDynamics_local/experiments/UPS/ups_29_1_21_updated_workflow_discovery"

# get bm
bm = UPSBenchmarker(experiment_home)

# get intensities and protein table
intensities = pd.read_csv(os.path.join(experiment_home, "Protein_Intensity.txt"), sep = "\t")
protein_table = pd.read_csv(os.path.join(experiment_home, "Protein_Table.txt"), sep = "\t")

In [91]:

# match ids and extract identifiers
intensities = intensities.merge(protein_table[['ProteinGroupId', 'ProteinId']], how = "left")
intensities['ProteinId'] = intensities.ProteinId.str.extract(r"([OPQ][0-9][A-Z0-9]{3}[0-9]|[A-NR-Z][0-9]([A-Z][A-Z0-9]{2}[0-9]){1,2})")[0]

#filter for non imputed and spiked proteins
spiked_proteins = bm.get_theoretical_logFCs().index.to_list()
filtered = intensities#[intensities.Imputed == 0]
filtered = filtered[filtered.ProteinId.apply(lambda x: x in spiked_proteins)]

# Get estimates in each condition
ups1 = filtered[filtered.condition == "UPS1"]
ups1 = ups1.groupby("ProteinId")["log2NInt_ProteinGroupId"].mean()
ups2 = filtered[filtered.condition == "UPS2"]
ups2 = ups2.groupby("ProteinId")["log2NInt_ProteinGroupId"].mean()

# get differences
dif = pd.DataFrame((ups1,ups2)).T
dif["estimated_fold_change"] = dif.iloc[:,0] - dif.iloc[:,1]


#get theoretical log2int
theoretical = bm.get_theoretical_logFCs()
dif = dif.merge(theoretical, left_index=True, right_index=True)
dif.columns = ["UPS1 Log2Int", "UPS2 Log2Int", "Dif Log2Int Estimated", "Dif Log2Int Theoretical"]

In [92]:
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr
import plotly.graph_objects as go

tmp = dif[~dif.isna().any(axis = 1)]
ests = tmp["Dif Log2Int Estimated"]
preds = tmp["Dif Log2Int Theoretical"]

fig = go.Figure()
fig.add_trace(go.Scatter(x=[-15, 15], y=[-15, 15], mode="lines",
                            line=go.scatter.Line(
                                color="gray", dash="dashdot"),
                            showlegend=False))

fig.add_trace(go.Scatter(x=preds, y=ests, 
                        mode='markers',
                        hovertext=dif.index))

reg = LinearRegression().fit(np.array(preds).reshape(-1, 1),np.array(ests))
print("Intercept of a Linear Regression: ", reg.intercept_)
print("Slope of a Linear Regression: ", reg.coef_)
print("R2 of a Linear Regression: ", reg.score(np.array(preds).reshape(-1, 1),np.array(ests)))
score = pearsonr(ests,preds)[0]
title = "PearsonR = " + str(round(score,3))
fig.update_layout(title= title,
                xaxis_title="True LogFC",
                yaxis_title="Estimated LogFC")

Intercept of a Linear Regression:  1.2416060294588704
Slope of a Linear Regression:  [0.50029793]
R2 of a Linear Regression:  0.8203602125541514


Protein table has passed integrity test.


In [21]:
filtered

,Run,ProteinGroupId,log2NInt_ProteinGroupId,Imputed,summarization,nRLE,z_norm,condition,replicate,run_id
0,1,0,31.657401,0,1,3.185743,0.923771,UPS1,2,UPS1.2
1,1,1,33.122026,0,1,2.666433,0.916965,UPS1,2,UPS1.2
2,1,2,32.892258,0,1,0.276345,0.790109,UPS1,2,UPS1.2
3,1,3,32.647908,0,1,2.697092,0.913409,UPS1,2,UPS1.2
4,1,4,31.466482,0,1,1.680895,0.904159,UPS1,2,UPS1.2
...,...,...,...,...,...,...,...,...,...,...
17193,8,3029,27.864209,0,1,0.008524,-0.018690,UPS2,3,UPS2.3
17195,8,3033,29.050300,0,1,1.082862,1.718106,UPS2,3,UPS2.3
17196,8,3035,29.624503,0,1,-0.043515,-0.313311,UPS2,3,UPS2.3
17197,8,3036,30.522595,0,1,-0.069083,-1.043398,UPS2,3,UPS2.3


0         True
1         True
2         True
3         True
4         True
         ...  
17193    False
17195    False
17196    False
17197    False
17199    False
Name: ProteinId, Length: 15958, dtype: bool